# Getting workspace

In [ ]:
from azureml.core import Workspace,Dataset,Environment,Experiment,ComputeTarget

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

# Getting predefined dataset, compute and environment

In [ ]:
#Getting raw dataset
raw_data=Dataset.get_by_name(ws,'Diabetes_Raw')
#Getting compute target
compute_instance = ComputeTarget(workspace=ws, name='EXTNAN1')
#Getting environment
environment = Environment.get(ws,'Diabetes-Model-Environment')


# Output dataset
OutputFileDatasetConfig can be used for temporary data output from pipeline steps and persistent output data. </br>
Docs: </br>
[OutputFileDataConfig docs](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.data.output_dataset_config.outputfiledatasetconfig?view=azure-ml-py) </br>
[Move data in pipelines](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-move-data-in-out-of-pipelines) </br>
[3 ways to pass data between pipeline steps](https://vladiliescu.net/3-ways-to-pass-data-between-azure-ml-pipeline-steps/)


In [ ]:
from azureml.data import OutputFileDatasetConfig
#Defining registered dataset as output
datastore=ws.get_default_datastore()
output_data1 = OutputFileDatasetConfig(destination = (datastore, 'aml_pipeline/{run-id}'))
#On completion of step register the output as a dataset
processed_dataset = output_data1.register_on_complete(name = 'Diabetes_pipeline_processed_data')

# Run configuration
Using predefined compute and envrionment

In [ ]:
from azureml.core.runconfig import RunConfiguration
aml_run_config = RunConfiguration()
aml_run_config.target = compute_instance
aml_run_config.environment=environment

# Defining steps

In [ ]:
from azureml.pipeline.steps import PythonScriptStep
#Input data definition
ds_input = raw_data.as_named_input('input_data')
# Defining data processing step
data_prep_step = PythonScriptStep(
    name='Data processing',
    script_name="./data_processing.py",
    source_directory="./scripts",
    arguments=["--input_data", ds_input,
            "--output", processed_dataset
            ],
    runconfig=aml_run_config,
    allow_reuse=True
)

In [ ]:
#Defining training step
train_step = PythonScriptStep(
  name='Training Model',
  script_name="./train.py",
  source_directory="./scripts",
  arguments=[
        "--input_data",
        processed_dataset.read_delimited_files().as_input(name='input_data'),
        "--model_name",
        "Pipeline-Model",
        "--target_column",
        "Binary_Target"
    ],
  runconfig=aml_run_config
)

# Combining steps into pipline

Docs:</br>
[How to use piplines](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-machine-learning-pipelines) </br>
[Code docs](https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipeline.pipeline?view=azure-ml-py)


In [ ]:
ml_pipeline_steps = [data_prep_step,train_step]

from azureml.pipeline.core import Pipeline

# Build the pipeline
pipeline1 = Pipeline(workspace=ws, steps=[ml_pipeline_steps])

In [ ]:
from azureml.core import Experiment

# Submit the pipeline to be run
pipeline_run1 = Experiment(ws, 'Diabetes_Pipeline').submit(pipeline1)
pipeline_run1.wait_for_completion()

# Publishing pipeline

Docs </br>
[How to](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-pipelines)

In [ ]:
published_pipeline1 = pipeline_run1.publish_pipeline(
     name="Diabetes_Pipeline",
     description="Published Pipeline Description",
     version="1.0")

# Triggering pipeline endpoint
In order to trigger the endpoint an AAD auth token has to be retrieved. Once That is done the pipline can be triggered as shown:
```
from azureml.pipeline.core import PublishedPipeline
import requests

response = requests.post(published_pipeline1.endpoint,
                         headers=aad_token,
                         json={"ExperimentName": "My_Pipeline",
                               "ParameterAssignments": {"pipeline_arg": 20}})
```
Docs: </br>
[Run published pipeline](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-pipelines#run-a-published-pipeline) </br>
[Authenticating in AML](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azureml.ipynb)